In [3]:
# compatibility with Python 2
from __future__ import print_function

# preprocess
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler, normalize
from sklearn.preprocessing import MinMaxScaler

# computes distance between each pair of two collections of inputs
from scipy.spatial.distance import pdist

# repaired matrix by averaging; probably the best can do without domain-specific information
from sklearn.utils.validation import check_symmetric

# clustering-related methods
from sklearn.cluster import SpectralClustering
from sklearn.cluster import KMeans
from sklearn.metrics import pairwise_distances
from sklearn.metrics import silhouette_score 

# dimensionality reduction
from sklearn.decomposition import PCA

# support plots
from ipywidgets import interact
import ipywidgets as widgets
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib import cm

import seaborn as sns; sns.set()
import networkx as nx

import numpy as np
import pandas as pd
import math

# To increase quality of figures
plt.rcParams["figure.figsize"] = (20, 7)

# display all the plots inline
%matplotlib inline

# fs-related operations
import os

# download files
import requests

# load *.arff files
from scipy.io.arff import loadarff

#float_formatter = lambda x: "%.3f" % x
#np.set_printoptions(formatter={'float_kind':float_formatter})

In [7]:
# before import the original file 'table1.xlsx' , deleted extra headings in excel
# relative directory not working for unknown reason
df = pd.read_excel('/Users/apple/PycharmProjects/Clustering/Modified Jupyter Notebooks/references/table1_fixed.xlsx')

In [8]:
# rename columns, for spaces sometimes rise errors in pandas
# r stands for residence, w for Place of Work
df.columns = ['r_state_fips', 'r_county_fips', 'r_state_name', 'r_county_name' , 
              'w_state_fips', 'w_county_fips', 'w_state_name', 'w_county_name', 'flow_score', 'margin_error']

# replace all NaN values with zeros (float)
df = df.fillna(0.0)

# delet the two descriptions rows in the end of the table:
df = df.drop(index = 139433)
df = df.drop(index = 139434)

# delet the row with fips value '301' & '303', which are Canada areas

In [9]:
# check datatype
df.dtypes

r_state_fips      object
r_county_fips    float64
r_state_name      object
r_county_name     object
w_state_fips     float64
w_county_fips    float64
w_state_name      object
w_county_name     object
flow_score       float64
margin_error     float64
dtype: object

In [10]:
# change the columns with FIPS codes into int
df[['r_state_fips', 'r_county_fips', 
    'w_state_fips', 'w_county_fips']] = df[['r_state_fips', 'r_county_fips', 
                                            'w_state_fips', 'w_county_fips']].astype(int)
df.head(5)

,r_state_fips,r_county_fips,r_state_name,r_county_name,w_state_fips,w_county_fips,w_state_name,w_county_name,flow_score,margin_error
0,1,1,Alabama,Autauga County,1,1,Alabama,Autauga County,8828.0,752.0
1,1,1,Alabama,Autauga County,1,13,Alabama,Butler County,6.0,10.0
2,1,1,Alabama,Autauga County,1,21,Alabama,Chilton County,504.0,228.0
3,1,1,Alabama,Autauga County,1,43,Alabama,Cullman County,27.0,44.0
4,1,1,Alabama,Autauga County,1,47,Alabama,Dallas County,296.0,130.0
5,1,1,Alabama,Autauga County,1,51,Alabama,Elmore County,2186.0,486.0
6,1,1,Alabama,Autauga County,1,53,Alabama,Escambia County,14.0,23.0
7,1,1,Alabama,Autauga County,1,73,Alabama,Jefferson County,271.0,142.0
8,1,1,Alabama,Autauga County,1,77,Alabama,Lauderdale County,8.0,16.0
9,1,1,Alabama,Autauga County,1,81,Alabama,Lee County,79.0,108.0


## Summarize the County "flow score" to its belonging State

In [17]:
# the flow score of a state is the sum of all its counties
# produce a pandas series
drop_county = df.groupby(['r_state_fips','w_state_fips'])['flow_score'].sum()

# convert the series into dataframe
state_flow = pd.DataFrame(drop_county, columns = ['flow_score'])
state_flow.to_csv('State_Mobility_FlowScore.csv')
state_flow.info()
state_flow.head(5)

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 2567 entries, (1, 0) to (72, 72)
Data columns (total 1 columns):
flow_score    2567 non-null float64
dtypes: float64(1)
memory usage: 26.1 KB


flow_score
r_state_fips w_state_fips            
1            0                  914.0
             1              1903899.0
             2                   99.0
             4                  146.0
             5                  285.0

## Spectral Clustering

The algorithm can be broken down into 4 basic steps.
1. Construct a similarity graph
2. Determine the Adjacency matrix W, Degree matrix D and the Laplacian matrix L
3. Compute the eigenvectors of the matrix L
4. Using the second smallest eigenvector as input, train a K-means model and use it to classify the data

## 1. Construct a similarity graph
Our dataset is composed of samples (rows) and thrie features (columns). However the spectral clustering algorithm can only be applied to a graph of connected nodes. 

### 1.1 Similarity / Affinity Matrix
So the first step here is to construct the **Similarity Matrix** from the original dataset. It is an NxN matrix where N is the number of samples. Then the cells should be filled with the euclidean distance between each pair of points. To calculate the euclidean distance, for loop will be too slow, while scipy.cdist will takes too much memory, scipy.pdist is more efficient. And sklearn.metrics.pairwise.cosine_similarity(X) also works.

However the flow score here is considered to be the distance between two States.

In [41]:
# unstack the multiindex dataframe
simi = state_flow.unstack()
simi = similarity.fillna(0.0)
simi.to_csv('State_Mobility_SimilarityMatrix.csv')
simi.head()

flow_score                                             \
w_state_fips        0          1         2          4          5     
r_state_fips                                                         
1                 914.0  1903899.0      99.0      146.0      285.0   
2                 222.0        0.0  355529.0       30.0        0.0   
4                1587.0       72.0     456.0  2730296.0      124.0   
5                 455.0      123.0     107.0      283.0  1190951.0   
6                8941.0      191.0     734.0     6599.0      327.0   

                                                      ...                \
w_state_fips         6       8      9     10     11   ...     49    50    
r_state_fips                                          ...                 
1                  515.0   164.0   18.0   0.0  239.0  ...    60.0  17.0   
2                   82.0    64.0    3.0  20.0   42.0  ...    38.0   0.0   
4                11208.0  1233.0  119.0  16.0  215.0  ...  2839.0  47.0   
5                  519.0   258.0    0.0   4.0   81.0  ...     9.0   0.0   
6             16786883.0  1587.0  469.0  78.0  856.0  ...  1373.0  41.0   

                                                                         
w_state_fips     51      53     54     55     56     72     301     303  
r_state_fips                                                             
1              526.0   185.0   57.0  157.0   71.0    0.0   31.0    76.0  
2               16.0   257.0    0.0    0.0   12.0    0.0    0.0     4.0  
4              240.0  1198.0   20.0  376.0  345.0    0.0  368.0  1164.0  
5              159.0    71.0   79.0  131.0   34.0    0.0   22.0     8.0  
6             1450.0  3818.0  154.0  563.0  143.0  126.0  968.0  3101.0  

[5 rows x 55 columns]

In [63]:
# convert pandas dataframe to a numpy ndarray by dropping the indexs
simiMtx = simi.values

# print the entire matrix (only this time)
with np.printoptions(threshold=np.inf):
    print(simiMtx)

[[9.1400000e+02 1.9038990e+06 9.9000000e+01 1.4600000e+02 2.8500000e+02
  5.1500000e+02 1.6400000e+02 1.8000000e+01 0.0000000e+00 2.3900000e+02
  8.7960000e+03 4.6500000e+04 5.2000000e+01 1.2000000e+01 3.4100000e+02
  2.8900000e+02 1.5000000e+02 7.1000000e+01 6.1400000e+02 2.9610000e+03
  3.7000000e+01 1.4000000e+02 2.6900000e+02 3.4900000e+02 2.2600000e+02
  1.3432000e+04 3.0000000e+02 5.1000000e+01 2.9000000e+01 4.8000000e+01
  8.0000000e+00 1.9500000e+02 6.2000000e+01 3.8500000e+02 6.4400000e+02
  5.0000000e+01 4.4000000e+02 1.4000000e+02 6.6000000e+01 4.8400000e+02
  0.0000000e+00 4.6200000e+02 8.0000000e+00 8.2810000e+03 2.1210000e+03
  6.0000000e+01 1.7000000e+01 5.2600000e+02 1.8500000e+02 5.7000000e+01
  1.5700000e+02 7.1000000e+01 0.0000000e+00 3.1000000e+01 7.6000000e+01]
 [2.2200000e+02 0.0000000e+00 3.5552900e+05 3.0000000e+01 0.0000000e+00
  8.2000000e+01 6.4000000e+01 3.0000000e+00 2.0000000e+01 4.2000000e+01
  1.7000000e+01 5.6000000e+01 2.3000000e+01 1.0000000e+01 0.000

In [60]:
# check the number of axes (dimensions) of the array
print(simiMtx.ndim)
print(simiMtx.shape)

2
(52, 55)


In [115]:
# similarity / affinity matrix must be 2-dimensional and square shape
# drop columns with fips = 0, 72, 301, 303, which is column 1, 53, 54, and 55.
simiMtx_square = np.delete(simiMtx, 1, axis = 1)
simiMtx_square = np.delete(simiMtx, np.s_[51:54], axis = 1)

print(simiMtx_square.ndim)
print(simiMtx_square.shape)

2
(52, 52)


In [116]:
# when first run the sklearn.SpectralClustering model
# UserWarning: Array is not symmetric, and will be converted to symmetric by average with its transpose.
# warnings.warn("Array is not symmetric, and will be converted")
# However all the values in simiMtx_square are positive numbers. 
print(np.any(np.isnan(simiMtx_square)))
print(np.any(np.isinf(simiMtx_square)))

False
False


In [118]:
# repaired matrix by averaging; probably the best can do without domain-specific information
from sklearn.utils.validation import check_symmetric

# default: raise_warning = True, raise_exception = False
simiMtx_square_repaired = check_symmetric(simiMtx_square) 
print('max error: ', np.amax(np.abs(simiMtx_square - simiMtx_square.T)))        
print('max error repaired: ', np.amax(simiMtx_square_repaired - simiMtx_square_repaired.T))

max error:  16786769.0
max error repaired:  0.0


## Spectral Clustering with sciki-learn

The similarity matrix acquired in the previous steps is the affinity matrix we need for the **`SpectralClustering`** function. The function has a parameter called **affinity** (string or callable, default **`rbf`**), set it as 

**`precomputed`** : interpret X as a precomputed affinity matrix.

Only kernels that produce similarity scores (non-negative values that increase with similarity) should be used. This property is not checked by the clustering algorithm.

In [124]:
# spectral-cluster the states into 10 regions with default core
# build the model 
spectral_model_kmeans = SpectralClustering(n_clusters = 10, affinity ='precomputed') 
  
# train the model and store the predicted cluster labels 
sc_label = spectral_model_kmeans.fit_predict(simiMtx_square_repaired) 
sc_label

array([3, 3, 1, 1, 1, 1, 1, 1, 1, 8, 8, 8, 9, 9, 9, 9, 9, 9, 2, 2, 2, 2,
       2, 2, 2, 0, 0, 0, 0, 0, 7, 7, 7, 7, 7, 5, 5, 5, 5, 5, 5, 5, 6, 6,
       6, 6, 4, 4, 4, 4, 4, 6], dtype=int32)

*Reference*

https://towardsdatascience.com/unsupervised-machine-learning-spectral-clustering-algorithm-implemented-from-scratch-in-python-205c87271045

https://www.geeksforgeeks.org/ml-spectral-clustering/

https://scikit-learn.org/stable/modules/generated/sklearn.cluster.SpectralClustering.html#sklearn-cluster-spectralclustering

https://towardsdatascience.com/spectral-clustering-for-beginners-d08b7d25b4d8

https://towardsdatascience.com/spectral-graph-clustering-and-optimal-number-of-clusters-estimation-32704189afbe

https://blog.csdn.net/a19990412/article/details/88672813

https://blog.csdn.net/weixin_36474809/article/details/89855869#3.1%20SpectralClustering

https://medium.com/acing-ai/what-is-cosine-similarity-matrix-f0819e674ad1

fix the similarity matrix:

https://stackoverflow.com/a/60575709

https://stackoverflow.com/a/38539257

https://stackoverflow.com/a/47800373

FIPS code: 

https://www.nrcs.usda.gov/wps/portal/nrcs/detail/?cid=nrcs143_013696

https://www.census.gov/prod/techdoc/cbp/cbp95/st-cnty.pdf

color map:

https://colorswall.com/palette/73/

cluster visualization:

https://www.kaggle.com/dhanyajothimani/basic-visualization-and-clustering-in-python